# <center></center>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential, backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras
import tensorflow_hub as hub
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import graphviz
%matplotlib inline
sns.set()

We will load data from data.npz file.

In [2]:
# Load all numpy arrays except training images (in order to save memory).
def load_data():
    with np.load('data.npz', allow_pickle=True) as npz_file:
        #X_train = npz_file['X_train']
        X_valid = npz_file['X_valid']
        X_test = npz_file['X_test']
        X_train_features = npz_file['X_train_features']
        X_valid_features = npz_file['X_valid_features']
        X_test_features = npz_file['X_test_features']
        y_train_1h = npz_file['y_train_1h']
        y_valid_1h = npz_file['y_valid_1h']
        y_test_1h = npz_file['y_test_1h']
        y_train = npz_file['y_train']
        y_valid = npz_file['y_valid']
        y_test = npz_file['y_test']
        class_indices = npz_file['class_indices']
        train_filenames = npz_file['train_filenames']
        valid_filenames = npz_file['valid_filenames']
        test_filenames = npz_file['test_filenames']
    return X_train_features, y_train_1h, y_train, train_filenames, X_valid,X_valid_features, y_valid_1h, y_valid, valid_filenames, X_test, X_test_features, y_test_1h, y_test, test_filenames,class_indices

# load training images
def load_images():
    with np.load('data.npz', allow_pickle=True) as npz_file:
        X_train = npz_file['X_train']
    return X_train

# merge two dictionaries
def merge_dict(x,y):
    d = x.copy()
    d.update(y)
    return d

# fit a model
def fit_model(model, grid, X_tr, y_tr, X_va, y_va, X_t, y_t):
    valid_scores = []
    train_scores = []
    params = []
    for params_dict in grid:
        #print (params_dict)
        params.append(params_dict)
        model.set_params(**params_dict)
        model.fit(X_tr, y_tr)
        train_scores.append(model.score(X_tr, y_tr))
        valid_scores.append(model.score(X_va, y_va))
    best_index = np.argmax(valid_scores)
    # refit model with best params
    model.set_params(**params[best_index])
    model.fit(X_tr, y_tr)
    test_score = model.score(X_t, y_t)
    return { 'best_params':params[best_index],
              'params':params,
             'train_scores':train_scores,
             'valid_scores':valid_scores,
             'test_score':test_score,
             'best_index':best_index,
              'best_valid_score':valid_scores[best_index],
              'best_model':model
            }

X_train_features, y_train_1h, y_train, train_filenames, X_valid,X_valid_features, y_valid_1h, y_valid, valid_filenames, X_test, X_test_features, y_test_1h, y_test, test_filenames,class_indices = load_data()

X_train = load_images()


## Convolutional neural network
You tested above different models with the set of high-level features extracted from a
pretrained neural network. However, can you get similar results by (re)training a
ConvNet from the pixels?
- What accuracy can you achieve?

We will implement a simple ConvNet because we have very small training dataset :

- conv2D layer : 16 kernels of size 5x5, 2x2 stride, "same" padding and ReLU activation
- MaxPooling2D : 2x2 size and stride
- conv2D layer : 16 kernels of size 3x3, 1x1 stride, "same" padding and ReLU activation
- MaxPooling2D : 2x2 size and stride
- Reshaping of the last layer to a 1-dimensional flat array.
- Fully-connected layer with 2048 outputs.
- Fully-connected layer with 6 outputs.

In [3]:
X_train.shape

(1400, 299, 299, 3)

In [4]:
model_cnn = Sequential()
model_cnn.add(keras.layers.Conv2D(16, kernel_size=(5, 5), strides=(2, 2),padding='same', activation='relu',
                 input_shape=(299,299,3)))
model_cnn.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_cnn.add(keras.layers.Conv2D(16, (3, 3), strides=(1, 1),padding='same',activation='relu'))
model_cnn.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_cnn.add(keras.layers.Flatten())
model_cnn.add(Dense(2048, activation='relu'))
model_cnn.add(Dense(6, activation='softmax'))
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 21904)             0         
_________________________________________________________________
dense (Dense)                (None, 2048)              44861440  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 12294     
Total para

We will compile and train it.

In [5]:
np.random.seed(0)
tf.set_random_seed(0)
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
cnn_fit = model_cnn.fit(
                        x=X_train, y=y_train_1h,
                        validation_data=(X_valid, y_valid_1h), batch_size=32, epochs=50,
                        callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)]
                        ,shuffle=True
                        )
(test_loss, test_accuracy) = model_cnn.evaluate(X_test, y_test_1h, batch_size=100)
print("Test Accuracy :", test_accuracy)

Train on 1400 samples, validate on 139 samples
Epoch 1/50
1400/1400 [==============================] - 49s 35ms/step - loss: 2.8584 - acc: 0.2186 - val_loss: 1.6761 - val_acc: 0.3309
Epoch 2/50
1400/1400 [==============================] - 42s 30ms/step - loss: 1.5727 - acc: 0.3821 - val_loss: 1.4481 - val_acc: 0.4460
Epoch 3/50
1400/1400 [==============================] - 37s 27ms/step - loss: 1.2816 - acc: 0.5264 - val_loss: 1.3879 - val_acc: 0.4892
Epoch 4/50
1400/1400 [==============================] - 37s 26ms/step - loss: 0.7423 - acc: 0.7600 - val_loss: 1.3249 - val_acc: 0.5036
Epoch 5/50
1400/1400 [==============================] - 39s 28ms/step - loss: 0.2898 - acc: 0.9236 - val_loss: 1.6353 - val_acc: 0.4964
Epoch 6/50
1400/1400 [==============================] - 36s 25ms/step - loss: 0.0608 - acc: 0.9900 - val_loss: 1.9769 - val_acc: 0.4964
Epoch 7/50
1400/1400 [==============================] - 35s 25ms/step - loss: 0.0127 - acc: 0.9993 - val_loss: 2.1383 - val_acc: 0.4604
E

We will compute it's accuracy on our test set.

In [6]:
(test_loss, test_accuracy) = model_cnn.evaluate(X_test, y_test_1h, batch_size=100)
print("Test Accuracy :", test_accuracy)

50/50 [==============================] - 0s 6ms/step
Test Accuracy : 0.5600000023841858


Test accuracy is very low.

- Can you get good results? - If not, why?

Our cnn model has about 45 million parameters! input space dimension is 299x299x3 = 268203! and our training set has only 1400 rows!

Training our model with few data will not generalize well. We need much more data to train our model.

Using transfer learning, as we have done in this project, is essential when we have very little data.